In [1]:
from torchkf import * 
import numpy as np
import scipy as sp
import sympy

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
""" format: value[, cov[, constaint]]
If constraint == 'positive', then the cov must be in log-space
""" 
parameters = dotdict(
    skeletal=dotdict(
        weight=dotdict(
            body=(70, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.00762, np.exp(-4), 'positive'),
                upper_arm=(0.01373, np.exp(-4), 'positive'), 
                hand=(0.00361, np.exp(-4), 'positive'),
            ), 
        ), 
        size=dotdict(
            body=(1.75, np.exp(0), 'positive'), 
            relative=dotdict(
                forearm=(0.16480, np.exp(-4), 'positive'),
                upper_arm=(0.19590, np.exp(-4), 'positive'), 
                hand=(0.11465, np.exp(-4), 'positive'),
                humerus_factor=(0.8, np.exp(-4), 'positive'),
            ), 
        ),
        constraints=dotdict(
            elbow=dotdict(
                max_flexion=np.pi / 6,
                max_extension=9 * np.pi / 10.,
            )
        ),
        g=9.81,
    ),
    muscles=dotdict(
        triceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2397.12, np.exp(2), 'positive'), # N
                Lm0  = (0.15, np.exp(-2), 'positive'), # m
                phi0 = (0.2094, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.18, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ),
        ), 
        biceps=dotdict(
            insertion=(0.02, np.exp(-8), 'positive'),
            tendon_length=(0.12, np.exp(-2), 'positive'),
            hill_muscle=dotdict(
                Fmax = (2874.67, np.exp(2), 'positive'), # N
                Lm0  = (0.13, np.exp(-2), 'positive'), # m
                phi0 = (0, np.exp(-2)), # rad
                W    = (0.56, np.exp(-2), 'positive'),
                vmax = (1.3, np.exp(-2), 'positive'), # m/s
                A    = 0.25, 
                gmax = 1.5, 
                Lslack = (0.21, np.exp(0), 'positive'), # m
                Kmee1 = 10, 
                Kmee2 = 10_000,
                Tact  = 0.1,
            ), 
        ),
    ), 
    afferents=dotdict(
        Ia=dotdict(
            kv=2.5,
            lmf=0.2,
            kdl=0.2,
            knl=0.5, 
            c1= 0.01,
        ), 
        Ib=dotdict(
            kf=1,
            fF=0.1,
        ),
    ),
)

In [8]:
def process_parameters(parameters): 
    proc_params = type(parameters)()
    for key, val in parameters.items(): 
        if isinstance(val, dict): 
            _proc_params = process_parameters(val)
            proc_params[key] = _proc_params
        else: 
            try: n = len(val)
            except TypeError: n = 0
            if n > 3: raise AttributeError(f'Parameter {key} has too many values (expected length 3)')
            param = [0, 0, None]
            
            # parameter value (prior expectation)
            if n == 0: param[0] = val
            else: param[0] = val[0]
                
            # parameter covariance
            if n > 1: param[1] = val[1]
                
            # parameter constraint
            if n == 3: param[2] = val[2]
            
            proc_params[key] = tuple(param)
    return proc_params
            
def flatten_parameters(parameters, namespace=''):
    flat_params = type(parameters)()
    for key, val in parameters.items(): 
        nkey = f'{namespace}.{key}' if len(namespace) > 0 else key
        if isinstance(val, dict): 
            _flat_params = flatten_parameters(val, namespace=nkey)
            flat_params.update(_flat_params)
        else: 
            flat_params[nkey] = val
    return flat_params            

In [10]:
import pandas as pd
df = pd.DataFrame(flatten_parameters(process_parameters(parameters))).transpose()
df = df.set_axis(['expectation', 'covariance', 'constraint'], axis=1)
df['constraint'] = df['constraint'].apply(lambda x: x if isinstance(x, str) else 'none')
df['covariance'] = df['covariance'].apply(lambda x: f'exp({int(np.log(x))})' if x > 0 else 0)
df

,expectation,covariance,constraint
skeletal.weight.body,70,exp(0),positive
skeletal.weight.relative.forearm,0.00762,exp(-4),positive
skeletal.weight.relative.upper_arm,0.01373,exp(-4),positive
skeletal.weight.relative.hand,0.00361,exp(-4),positive
skeletal.size.body,1.75,exp(0),positive
skeletal.size.relative.forearm,0.1648,exp(-4),positive
skeletal.size.relative.upper_arm,0.1959,exp(-4),positive
skeletal.size.relative.hand,0.11465,exp(-4),positive
skeletal.size.relative.humerus_factor,0.8,exp(-4),positive
skeletal.constraints.elbow.max_flexion,0.523599,0,none


In [28]:
from abc import ABC, abstractmethod

class NamedParamModel(ABC): 
    def __init__(self, 
                 state_names, input_names, param_names, 
                 state_map=None, input_map=None, param_map=None,
                 state_keys=None, input_keys=None, param_keys=None):
        
        self.state_names = state_names
        self.input_names = input_names
        self.param_names = param_names
        
        if state_map is None: 
            self.state_map = {k:k for k in self.state_names}
        else: 
            self.state_map = {
                k:state_map[k] if k in state_map.keys() else k 
                for k in state_names}
        
        if input_map is None: 
            self.input_map = {k:k for k in self.input_names}
        else: 
            self.input_map = {
                k:input_map[k] if k in input_map.keys() else k 
                for k in input_names}
            
        if param_map is None: 
            self.param_map = {k:k for k in self.param_names}
        else: 
            self.param_map = {
                k:param_map[k] if k in param_map.keys() else k 
                for k in param_names}
        
        if None not in (state_keys, input_keys, param_keys): 
            self.compute_indices(state_keys, input_keys, param_keys)
        
    def compute_indices(self, state_keys, input_keys, param_keys):
        self.state_keys = state_keys
        self.input_keys = input_keys
        self.param_keys = param_keys
        
        self.ix = [state_keys.index(self.state_map[k]) for k in self.state_names]
        self.iv = [input_keys.index(self.input_map[k]) for k in self.input_names]
        self.ip = [param_keys.index(self.param_map[k]) for k in self.param_names]
        
    @abstractmethod
    def compute_output(self, *args, **kwargs): 
        pass
    
    def __call__(self, x, v, p): 
        _x = x[:, 0][self.ix]
        _v = v[:, 0][self.iv]
        _p = p[:, 0][self.ip]
        
        return self.compute_output(*_x, *_v, *_p)
    
    def check_model(self, n=None, m=None, nP=None): 
        if n  is None: n  = len(self.state_keys)
        if m  is None: m  = len(self.input_keys)
        if nP is None: nP = len(self.param_keys)
            
        x = sympy.MatrixSymbol('x', n, 1)
        v = sympy.MatrixSymbol('v', m, 1)
        p = sympy.MatrixSymbol('p', nP, 1)
        
        x = np.asarray(x)
        v = np.asarray(v)
        p = np.asarray(p)
        
        print('Checking model... ')
        y = self.__call__(x,v,p)
        try: 
            l, l2 = y.shape
            assert(l2 == 1)
            print(' return shape ok.')
        except: 
            print(' error: function should return a vector of size (l, 1)')
            
        print('Computing derivatives... ')
        df, d2f = compute_sym_df_d2f(triceps_muscle, n, m, nP)
        print('Checkup passed!')

In [29]:
class HillMuscle(NamedParamModel):  
    state_names = 'L', 'vel'
    input_names = 'aMN', 'gMN' 
    param_names = 'Kmee1', 'Kmee2', 'Lslack', 'Fmax', 'Lm0', 'W', 'vmax', 'A', 'gmax', 'phi0', 'kv', 'kdl', 'knl', 'c1', 'kf'
        
    def __init__(self, *args, **kwargs): 
        super().__init__(HillMuscle.state_names, HillMuscle.input_names, HillMuscle.param_names, *args, **kwargs)
    
    @staticmethod
    def force_velocity(vel, vmax, A, gmax): 
        cd    = vmax * A * (gmax - 1) / (A + 1.)
        expr1 = (vmax + vel) / (vmax - vel/A)
        expr2 = (gmax * vel + cd) / (vel + cd)
        
        return sympy.Piecewise((expr1, vel <= 0), (expr2, True))
    
    @staticmethod
    def force_length(L, Lm0, W): 
        return sympy.exp(- (L - Lm0)**2/(W * Lm0)**2)
    
    @staticmethod
    def Fmee(L, Kmee1, Kmee2, Lslack):
        expr1 = Kmee1 * (L - Lslack)
        expr2 = Kmee2 * (L - Lslack)**2
        
        return sympy.Piecewise((expr1, (L < Lslack)), (expr1 + expr2, True))
    
    @staticmethod
    def Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax): 
        fl = HillMuscle.force_length(L, Lm0, W)
        fv = HillMuscle.force_velocity(vel, vmax, A, gmax)
        
        return aMN * Fmax * fl * fv
    
    @staticmethod
    def Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, vmax, A, gmax, phi0):     
        Fmee = HillMuscle.Fmee(L, Kmee1, Kmee2, Lslack)
        Fmce = HillMuscle.Fmce(aMN, Fmax, L, vel, Lm0, W, vmax, A, gmax)
        
        return (Fmee + Fmce) * sympy.cos(Lm0 * sympy.sin(phi0) / L)

    @staticmethod
    def rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1):
        dnorm = (L - 0.2 * Lm0) / Lm0
        vnorm = vel / vmax
        Ia    = kv * vnorm + kdl * dnorm + knl * gMN + c1 
        Ia    = sympy.Piecewise((0, Ia < 0), (Ia, Ia <= 1), (1, True))
        return Ia
    
    @staticmethod
    def rateIb(Frte, Fmax, kf): 
        Fnorm = (Frte - 0.1 * Fmax) / Fmax
        Ib    = kf * Fnorm
        
        Ib    = sympy.Piecewise((0, Ib < 0), (Ib, Ib <=1), (1, True))
        return Ib
    
    def compute_output(self, L, vel, aMN, gMN, Kmee1, Kmee2, Lslack, Fmax, Lm0, W, vmax, A, gmax, phi0, kv, kdl, knl, c1, kf):      
        
        Frte   = HillMuscle.Frte(L, vel, aMN, Kmee1, Kmee2, Lslack, Fmax, W, vmax, A, gmax, Lm0, phi0)
        rateIa = HillMuscle.rateIa(L, vel, gMN, Lm0, vmax, kv, kdl, knl, c1)
        rateIb = HillMuscle.rateIb(Frte, Fmax, kf)       
        
        return sympy.Matrix([[Frte], [rateIa], [rateIb]])

In [30]:
flatten_parameters(process_parameters(parameters))

{'skeletal.weight.body': (70, 1.0, 'positive'),
 'skeletal.weight.relative.forearm': (0.00762,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.upper_arm': (0.01373,
  0.01831563888873418,
  'positive'),
 'skeletal.weight.relative.hand': (0.00361, 0.01831563888873418, 'positive'),
 'skeletal.size.body': (1.75, 1.0, 'positive'),
 'skeletal.size.relative.forearm': (0.1648, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.upper_arm': (0.1959, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.hand': (0.11465, 0.01831563888873418, 'positive'),
 'skeletal.size.relative.humerus_factor': (0.8,
  0.01831563888873418,
  'positive'),
 'skeletal.constraints.elbow.max_flexion': (0.5235987755982988, 0, None),
 'skeletal.constraints.elbow.max_extension': (2.827433388230814, 0, None),
 'skeletal.g': (9.81, 0, None),
 'muscles.triceps.insertion': (0.02, 0.00033546262790251185, 'positive'),
 'muscles.triceps.tendon_length': (0.12, 0.1353352832366127, 'positive'),
 'mus

In [31]:
state_keys = 'L', 'vel'
input_keys = 'aMN', 'gMN'
params     = flatten_parameters(process_parameters(parameters))
param_keys = [*params.keys()]
param_map  = {k: f'muscles.triceps.hill_muscle.{k}' 
              for k in HillMuscle.param_names if f'muscles.triceps.hill_muscle.{k}' in param_keys}
param_map.update(
    {k: f'afferents.Ia.{k}' for k in HillMuscle.param_names if f'afferents.Ia.{k}' in param_keys})
param_map.update(
    {k: f'afferents.Ib.{k}' for k in HillMuscle.param_names if f'afferents.Ib.{k}' in param_keys})

triceps_muscle = HillMuscle(param_map=param_map ,state_keys=state_keys, input_keys=input_keys, param_keys=param_keys)

In [32]:
triceps_muscle.check_model()

Checking model... 
 return shape ok.
Computing derivatives... 
Checkup passed!
